In [9]:
!git clone https://github.com/muxspace/facial_expressions.git


Cloning into 'facial_expressions'...
remote: Enumerating objects: 14214, done.
remote: Total 14214 (delta 0), reused 0 (delta 0), pack-reused 14214
Receiving objects: 100% (14214/14214), 239.65 MiB | 42.35 MiB/s, done.
Resolving deltas: 100% (223/223), done.
Checking out files: 100% (13996/13996), done.


In [11]:
import csv
data={}
with open('/content/facial_expressions/data/legend.csv') as f:
  reader=csv.reader(f)
  next(reader)
  for row in reader:
    key=row[2].lower()
    if key in data:
      data[key].append(row[1])
    else:
      data[key]=[row[1]]


In [12]:
emotion_list=list(data.keys())
emotion_list

['anger',
 'surprise',
 'disgust',
 'fear',
 'neutral',
 'happiness',
 'sadness',
 'contempt']

In [13]:
import os

os.mkdir("master_data")
os.mkdir("master_data/training")
os.mkdir("master_data/testing")

In [14]:
for emotion in emotion_list:
  os.mkdir(os.path.join("master_data/training",emotion))
  os.mkdir(os.path.join("master_data/testing",emotion))

In [15]:
from shutil import copyfile
split_size=0.8

for emotion,images in data.items():
  train_size=int(split_size*len(images))
  train_images=images[:train_size]
  test_images=images[train_size:]
  for image in train_images:
    source=os.path.join("/content/facial_expressions/images",image)
    des=os.path.join("/content/master_data/training",emotion,image)
    copyfile(source,des)
  for image in test_images:
    source=os.path.join("/content/facial_expressions/images",image)
    des=os.path.join("/content/master_data/testing",emotion,image)
    copyfile(source,des)

In [16]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten


In [27]:
model=tf.keras.models.Sequential([
      Conv2D(16,(3,3),activation='relu',input_shape=(100,100,3)),
      MaxPooling2D(2,2),
      Conv2D(32,(3,3),activation='relu'),
      MaxPooling2D(2,2),
      Conv2D(64,(3,3),activation='relu'),
      MaxPooling2D(2,2),
      Flatten(),
      Dense(256,activation='relu'),
      Dense(8,activation='softmax')
])
model.compile(optimizer= keras.optimizers.Adam(learning_rate=0.01), loss='categorical_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 98, 98, 16)        448       
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 49, 49, 16)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 47, 47, 32)        4640      
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 21, 21, 64)        18496     
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 10, 10, 64)        0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 6400)             

In [24]:
train_dir='/content/master_data/training'
test_dir='/content/master_data/testing'

train_datagen=ImageDataGenerator(rescale=1.0/255)
train_generator=train_datagen.flow_from_directory(
    train_dir,
    target_size=(100,100),
    class_mode='categorical',
    batch_size=128
)

test_datagen=ImageDataGenerator(rescale=1.0/255)
test_generator=test_datagen.flow_from_directory(
    test_dir,
    target_size=(100,100),
    class_mode='categorical',
    batch_size=128
)

Found 10941 images belonging to 8 classes.
Found 2742 images belonging to 8 classes.


In [28]:
es=EarlyStopping(monitor='val_acc',patience=2,min_delta=0.01)

In [29]:
model.fit_generator(train_generator,
                    epochs=10,
                    verbose=1,
                    validation_data=test_generator,
                    callbacks=[es]
                    )

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
86/86 [==============================] - 15s 174ms/step - loss: 2.2426 - acc: 0.4623 - val_loss: 1.0837 - val_acc: 0.5011
Epoch 2/10
86/86 [==============================] - 15s 171ms/step - loss: 0.9320 - acc: 0.6180 - val_loss: 1.0509 - val_acc: 0.6065
Epoch 3/10
86/86 [==============================] - 15s 171ms/step - loss: 0.7729 - acc: 0.7286 - val_loss: 1.0489 - val_acc: 0.6331
Epoch 4/10
86/86 [==============================] - 15s 172ms/step - loss: 0.6747 - acc: 0.7684 - val_loss: 1.1040 - val_acc: 0.6455
Epoch 5/10
86/86 [==============================] - 15s 172ms/step - loss: 0.6408 - acc: 0.7770 - val_loss: 1.2295 - val_acc: 0.6200
Epoch 6/10
86/86 [==============================] - 15s 171ms/step - loss: 0.6140 - acc: 0.7834 - val_loss: 1.3897 - val_acc: 0.6426
